<a href="https://colab.research.google.com/github/ambideXtrous9/ECG-Keras/blob/main/ECG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import keras.layers as layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import random

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/MyDrive/ECG/data.csv'


In [4]:
df = pd.read_csv(path)
df


,0,1,2,3,4,5,6,7,8,9,...,202,203,204,205,206,207,208,209,210,211
0,-0.285,-0.275,-0.285,-0.290,-0.280,-0.300,-0.320,-0.360,-0.385,-0.455,...,-0.385,-0.395,-0.410,-0.395,-0.405,-0.385,-0.390,-0.405,100,0
1,-0.335,-0.315,-0.300,-0.295,-0.285,-0.255,-0.250,-0.255,-0.230,-0.245,...,-0.335,-0.330,-0.345,-0.345,-0.330,-0.345,-0.345,-0.330,100,0
2,-0.370,-0.360,-0.340,-0.315,-0.330,-0.320,-0.300,-0.320,-0.315,-0.305,...,-0.350,-0.365,-0.355,-0.370,-0.370,-0.365,-0.365,-0.370,100,0
3,-0.390,-0.360,-0.335,-0.335,-0.325,-0.305,-0.315,-0.305,-0.300,-0.310,...,-0.335,-0.335,-0.350,-0.340,-0.335,-0.335,-0.335,-0.340,100,0
4,-0.320,-0.320,-0.315,-0.285,-0.295,-0.285,-0.270,-0.290,-0.295,-0.280,...,-0.315,-0.330,-0.330,-0.335,-0.330,-0.315,-0.335,-0.345,100,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107180,2.395,2.360,2.355,2.350,2.350,2.320,2.360,2.325,2.310,2.320,...,0.660,0.640,0.660,0.635,0.610,0.635,0.600,0.600,203,4
107181,0.865,0.880,0.860,0.840,0.865,0.815,0.810,0.820,0.810,0.785,...,-0.160,-0.180,-0.185,-0.175,-0.190,-0.220,-0.190,-0.230,203,4
107182,-0.005,-0.030,0.010,-0.025,-0.025,-0.005,-0.035,-0.030,-0.025,-0.045,...,-0.605,-0.570,-0.595,-0.625,-0.570,-0.615,-0.600,-0.595,203,4
107183,-0.870,-0.810,-0.795,-0.790,-0.765,-0.755,-0.770,-0.720,-0.705,-0.705,...,0.465,0.495,0.500,0.535,0.540,0.545,0.535,0.525,208,4


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107185 entries, 0 to 107184
Columns: 212 entries, 0 to 211
dtypes: float64(210), int64(2)
memory usage: 173.4 MB


In [6]:
df.shape

(107185, 212)

In [7]:
unique_values = df['210'].unique().tolist()


In [8]:
len(unique_values)

47

In [9]:
model = keras.Sequential([
    layers.Input(shape=[211]),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.1),
    layers.Dense(32, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.1),
    layers.Dense(5, activation='softmax')
])

In [10]:
model.compile(
    optimizer='adam',
    loss=keras.losses.CategoricalCrossentropy(),
    #loss='sparse_categorical_crossentropy'
    metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               108544    
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 batch_normalization_1 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 256)               0

In [11]:
def Model(n = 10,epochs=5,bs=64):
  hist = []
  for i in range(n):
    print("\n======================================Fold = {}========================================\n".format(i+1))
    chosen_values = random.sample(unique_values, k=9)

    mask = df['210'].isin(chosen_values)

    df_val = df[mask]
    df_train = df[~mask]

    
    X_train = df_train.iloc[:, :-1].values
    Y_train = df_train.iloc[:, -1].values
    
    X_val = df_val.iloc[:, :-1].values
    Y_val = df_val.iloc[:, -1].values

    valy = np.unique(Y_val)
    tray = np.unique(Y_train)

    print("valy = ",valy)
    print("tray = ",tray)

    if(len(valy) < 5 or len(tray) < 5):
      print("\n==============================STEP SKIPPED AS MIS-MTACHED CLASS==========================\n")
      continue


    X_train = tf.convert_to_tensor(np.array(X_train,dtype=np.float32))
    Y_train = tf.convert_to_tensor(np.array(Y_train,dtype=np.float32))
    X_val = tf.convert_to_tensor(np.array(X_val,dtype=np.float32))
    Y_val = tf.convert_to_tensor(np.array(Y_val,dtype=np.float32))

    # Assuming y_train is your target variable
    Y_train = to_categorical(Y_train)
    Y_val = to_categorical(Y_val)

    history = model.fit(X_train, Y_train,
                        validation_data=(X_val, Y_val),
                        batch_size=bs,
                        epochs=epochs)
    
    hist.append(pd.DataFrame(history.history))
  return pd.concat(hist, axis=0)

In [12]:
hist = Model(n = 10,epochs=10,bs=512)


======================================Fold = 1========================================

valy =  [0 1 2 3 4]
tray =  [0 1 2 3 4]
Epoch 1/10
171/171 [==============================] - 10s 39ms/step - loss: 0.6759 - accuracy: 0.8278 - val_loss: 0.9838 - val_accuracy: 0.7035
Epoch 2/10
171/171 [==============================] - 9s 50ms/step - loss: 0.2137 - accuracy: 0.9497 - val_loss: 0.4588 - val_accuracy: 0.8663
Epoch 3/10
171/171 [==============================] - 6s 37ms/step - loss: 0.1536 - accuracy: 0.9606 - val_loss: 0.4461 - val_accuracy: 0.8762
Epoch 4/10
171/171 [==============================] - 8s 48ms/step - loss: 0.1307 - accuracy: 0.9653 - val_loss: 0.7469 - val_accuracy: 0.8524
Epoch 5/10
171/171 [==============================] - 9s 55ms/step - loss: 0.1142 - accuracy: 0.9696 - val_loss: 0.7448 - val_accuracy: 0.8582
Epoch 6/10
171/171 [==============================] - 7s 41ms/step - loss: 0.1028 - accuracy: 0.9727 - val_loss: 0.7753 - val_accuracy: 0.8741
Epoch 7/10
1

In [13]:
mean_val_loss = hist['val_loss'].mean()
mean_val_accuracy = hist['val_accuracy'].mean()

In [14]:
print("mean_val_loss = ",mean_val_loss)
print("mean_val_accuracy = ",mean_val_accuracy)

mean_val_loss =  0.2823728890623897
mean_val_accuracy =  0.9214046642184257
